In [3]:
import backtrader as bt
import backtrader.plot
import backtrader.analyzers as btanalyzers
import matplotlib
import datetime
# import config, csv
from binance.client import Client
matplotlib.use('QT5Agg')

In [4]:
# SIMPLE MACD STRAT:
class RSIStrategy(bt.Strategy):

    def __init__(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash
        self.rsi = bt.talib.RSI(self.data, period=14)

    def next(self):
        if self.rsi < 30 and not self.position:
            self.buy(size=1)
        
        if self.rsi > 70 and self.position:
            self.close()
    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

    

In [6]:
class BuyAndHold(bt.Strategy):
    def start(self):
        self.val_start = self.broker.get_cash()  # keep the starting cash

    def nextstart(self):
        # Buy all the available cash
        size = int(self.broker.get_cash() / self.data)
        self.buy(size=size)

    def stop(self):
        # calculate the actual returns
        self.roi = (self.broker.get_value() / self.val_start) - 1.0
        print('ROI:        {:.2f}%'.format(100.0 * self.roi))

In [11]:
# client = Client(config.API_KEY, config.API_SECRET)

# csvfile = open('2020-21_5minutes.csv', 'w', newline='') 
# candlestick_writer = csv.writer(csvfile, delimiter=',')

# candlesticks = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_5MINUTE, "1 Jan, 2020", "27 May, 2021")
# #candlesticks = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "1 Jan, 2020", "12 Jul, 2020")
# #candlesticks = client.get_historical_klines("BTCUSDT", Client.KLINE_INTERVAL_1DAY, "1 Jan, 2017", "12 Jul, 2020")

# for candlestick in  candlesticks:
#     candlestick[0] = candlestick[0] / 1000
#     candlestick_writer.writerow(candlestick)

# csvfile.close()


In [7]:
cerebro = bt.Cerebro()

fromdate = datetime.datetime.strptime('2020-07-01', '%Y-%m-%d')
todate = datetime.datetime.strptime('2021-05-27', '%Y-%m-%d')

data = bt.feeds.GenericCSVData(dataname='2020-21_5minutes.csv', dtformat=2, compression=15, timeframe=bt.TimeFrame.Minutes, fromdate=fromdate, todate=todate)
print(data)
cerebro.adddata(data)

In [19]:
cerebro.addstrategy(RSIStrategy)
cerebro.broker.setcash(10000)
cerebro.addsizer(bt.sizers.PercentSizer, percents = 100)
cerebro.run()

cerebro.plot(iplot= False)

FileNotFoundError: [Errno 2] No such file or directory: '2020_5minutes.csv'

In [ ]:
cerebro.addstrategy(BuyAndHold)
cerebro.addsizer(bt.sizers.PercentSizer, percents = 100)
cerebro.run()

cerebro.plot(iplot= False)
